# About
* **Author**: Adil Rashitov
* **Created at**: 21.06.2021
* **Goal**: Prepare parameters for web scrappers [118 direct](http://www.118.direct)
* **Deliverable**: parameters in postgresSQL table

In [1]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Multiprocessing for Mac / Linux
import platform
platform.system()
if platform.system() == 'Darwin':
    from multiprocess import Pool
else:
    from multiprocessing import Pool

# Visualization libraries
import plotly.express as px

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False


import requests
from bs4 import BeautifulSoup
import re
import time

# GLOBAL VARS
from dotenv import load_dotenv
load_dotenv('./.rds_endpoint')

True

# Data

1. Web scrapping of businesses categories
2. Web scrapping of locations

In [2]:
S_CATEGORIES = '/WORKDIR/data/sources/bussiness_categories_to_extract/categories.csv.zip'
S_LOCATIONS = '/WORKDIR/data/sources/locations_to_extract/locations.csv.zip'


categs = pd.read_csv(S_CATEGORIES)
locs = pd.read_csv(S_LOCATIONS)

# Main

1. Generation of base urls for search
2. Extraction amount of pages from each category
3. Generation url with page specified to get all results
4. Export search parameters to database

### 1. Generation of base urls for search

In [3]:
# Generation of URLS
def container(func, kwargs):
    return func(**kwargs)


def generate_url(what: str, where: str) -> str:
    """
        Arguments:
            * what (str): business category name
            * where (str): location
    """
    return f"http://www.118.direct/listing/search?what={what}&where={where}"


def generate_dataframe_for_region(bussines_categs, loc):
    return pd.DataFrame({
        'what': bussines_categs,
        'where': loc
    })


search_params = generate_dataframe_for_region(categs['business_category'], locs.values[0][0])
search_params['url'] = \
    list(map(container,
             [generate_url]*search_params.shape[0],
             search_params.to_dict('record')))

search_params.head()

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,what,where,url
0,Abattoirs,Manchester,http://www.118.direct/listing/search?what=Abat...
1,Access Control Systems,Manchester,http://www.118.direct/listing/search?what=Acce...
2,Access Equipment,Manchester,http://www.118.direct/listing/search?what=Acce...
3,Accountants,Manchester,http://www.118.direct/listing/search?what=Acco...
4,Accounting & Bookkeeping Services,Manchester,http://www.118.direct/listing/search?what=Acco...


### 2. Extraction amount of pages from each category

In [4]:
def extract_n_entities(arg: dict) -> dict:
    """
        Gets amount of entities available
        for web scrapping at particular location.
        
        Arguments:
            * arg (dict): request arguments
                * `url`: url of specified category and locations
    """
    time.sleep(np.random.normal(1, 0.2))

    html_text = requests.get(arg['url']).text
    try:
        soup = BeautifulSoup(html_text, "html.parser")
        n_entities = soup \
            .find_all("div", class_="resultInfoBlock")[0] \
            .contents[0]

        n_entities = re.findall(r'\d+', n_entities.split('of ')[1])
        arg['n_entities'] = int(n_entities[0])

    except Exception as exc:
        logging.error(f"{arg}: {str(exc)}")

    return arg

In [5]:
!mkdir -p /WORKDIR/data/intermediate/request_params_with_n_entities_in_category/

In [6]:
# N entites in category
STEP2_TARGET_FILE = '/WORKDIR//data/intermediate/request_params_with_n_entities_in_category/parameters.csv.zip'

if not os.path.isfile(STEP2_TARGET_FILE):
    logging.info("Sending requests to extract n entities available")
    with Pool(2) as pool:
        search_params = pool.starmap(extract_n_entities,
                                     zip(search_params.to_dict('record')))
    del pool
    search_params = pd.DataFrame(search_params).reset_index(drop=True)
    
    search_params.to_csv('/WORKDIR/data/intermediate/request_params_with_n_entities_in_category/parameters.csv.zip', index=False, compression='zip')
else:
    logging.info("File already exsits...")
    logging.info(f"Reading from {STEP2_TARGET_FILE}...")
    search_params = pd.read_csv(STEP2_TARGET_FILE)

[ 06/25/2021 03:05:59 PM ][ INFO ]: File already exsits...
[ 06/25/2021 03:05:59 PM ][ INFO ]: Reading from /WORKDIR//data/intermediate/request_params_with_n_entities_in_category/parameters.csv.zip...


### 3. Generation url with page specified to get all results

Each page has 15 entities, so we need to map range of entites to page to know page urls

* **input format**: `http://www.118.direct/listing/search?what=<what>&where=<where>`
* **output format**:`http://www.118.direct/listing/search?what=<what>&where=<where>&page=<page>`

In [7]:
# Generation url with page parameter
n_entities_in_list = 15

# 1. Extraction n pages for each category
n_int_pages = search_params['n_entities'] // n_entities_in_list
is_div_fractional = (search_params['n_entities'] % n_entities_in_list) > 0
search_params['n_pages'] = n_int_pages + is_div_fractional


# 2. Generation urls for pages
def generate_url_with_pages(arg):
    return pd.DataFrame({
        **arg,
        'page': list(range(1, arg['n_pages']+1, 1))
    })

search_params_with_page = list(map(
    generate_url_with_pages,
    search_params.to_dict('records')
))

logging.info(f"N urls at input: {search_params.shape[0]}")
search_params_with_page = pd.concat(search_params_with_page).reset_index(drop=True)
col_rename_params = {
    'url': 'base_url',
    'n_entities': 'n_entities_at_category_at_location',
    'n_pages': 'n_pages_at_category_at_location',
    'page': 'page'
}
search_params_with_page = search_params_with_page.rename(columns=col_rename_params)
logging.info(f"N urls at output: {search_params_with_page.shape[0]}")

# 3. Building full url
search_params_with_page['full_url'] = search_params_with_page['base_url'] + "&page=" + search_params_with_page['page'].astype(str)


# 4. Columns selection
search_params_with_page['scrapping_parameter_id'] = list(search_params_with_page.index)
cols = [
    'scrapping_parameter_id', 'what', 'where', 'n_entities_at_category_at_location',
    'n_pages_at_category_at_location', 'page', 'base_url', 'full_url'
]
search_params_with_page = search_params_with_page[cols]

[ 06/25/2021 03:06:06 PM ][ INFO ]: N urls at input: 1151
[ 06/25/2021 03:06:06 PM ][ INFO ]: N urls at output: 6638


### 4. Export search parameters to database

Export request parameters to rds database

In [8]:
import sqlalchemy

TABLE_NAME="WEB_SCRAPPER_PARAMETERS"

RDS_ENGINE = sqlalchemy.create_engine(os.environ['RDS_ENDPOINT'])


WEB_SCRAPPER_PARAMETERS_SCHEMA = {
    'scrapping_parameter_id': sqlalchemy.types.Integer,
    'what': sqlalchemy.types.String,
    'where': sqlalchemy.types.String,
    'n_entities_at_category_at_location': sqlalchemy.types.SmallInteger,
    'n_pages_at_category_at_location': sqlalchemy.types.SmallInteger,
    'page': sqlalchemy.types.SmallInteger,
    'base_url': sqlalchemy.types.String,
    'full_url': sqlalchemy.types.String
}

In [9]:
search_params_with_page.to_sql(
    name=TABLE_NAME,
    con=RDS_ENGINE,
    schema="public",
    if_exists="replace",
    index=False,
    dtype=WEB_SCRAPPER_PARAMETERS_SCHEMA,
    method="multi"
)